<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/fine-tuning-experiments/askpatient/askApatient_8_tuning_bigtweet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
os.mkdir(base_dir)

In [0]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   287  100   287    0     0   1017      0 --:--:-- --:--:-- --:--:--  1021
bash: line 1: syntax error near unexpected token `newline'
bash: line 1: `<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">'


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# training data set

train_csv = path/'AskAPatient.fold-8.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [47]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-8.validation.csv",header=None)
valid.head()

,0,1
0,0,scared to death
1,2,heel pain
2,5,increased heart rate
3,6,stage 1 diastolic dysfunction
4,8,arthritis


In [48]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-8.test.csv",header=None)
test.head()

,0,1
0,1,Abdominal rash
1,5,increase my heart rate
2,5,tachycardia
3,8,arthritis
4,8,arthritis


In [0]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'fine-tuning_language_model_bigtweet_data.csv')

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas.add_test(test,label=0)


In [53]:
data_lm.save()
data_clas.save()

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type CrossEntropyLoss. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [54]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'rt',
 'to',
 'the',
 'for',
 'a',
 'in',
 'you',
 'is',
 'of',
 'on',
 'and',
 'disease',
 'amp',
 'i',
 'gluten',
 'with',
 'your',
 'free',
 'it',
 'my',
 'have',
 'are',
 'new',
 'this',
 'about',
 'glutenfree',
 'our',
 'what',
 'nt',
 'do',
 'celiac',
 'out',
 'from',
 'be',
 'at',
 'now',
 'not',
 'or',
 'that',
 'we',
 'how',
 'food',
 'can',
 'via',
 'its',
 'right',
 'tip',
 'more',
 'all',
 'if',
 'use',
 'diet',
 'dblclick',
 'w',
 'pic',
 'so',
 'products',
 'as',
 'why',
 'review',
 'check',
 'up',
 'know',
 'here',
 'love',
 'me',
 'by',
 'get',
 'just',
 'gfb',
 'thanks',
 'who',
 'help',
 'but',
 're',
 'an',
 'could',
 'posted',
 's',
 'm',
 'post',
 'recipe',
 'will',
 'was',
 'has',
 'great',
 'need',
 'may',
 'intolerance',
 'heres',
 'us',
 'read',
 'archives',
 'people',
 'make',
 '\r \n ',
 'does',
 'some',
 'eat',
 'like',
 'there',
 'living',
 'test',
 'life',
 'when',
 

In [0]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [57]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,5.107330,4.237797,0.267474,03:59


In [58]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.032104,3.710143,0.328093,03:57


In [59]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.734391,3.672119,0.335427,03:56


In [60]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.697446,3.637869,0.339541,03:55


In [0]:
learn.save_encoder('bigtweet_fold8_first')

In [62]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('bigtweet_fold8_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.610976,4.457202,0.320592,01:07


In [63]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.832397,4.013821,0.344020,01:20


In [64]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.518799,3.834832,0.352651,03:12


In [65]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.382530,3.615610,0.369914,03:26
1,3.209239,3.554202,0.392109,03:09


In [66]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.090823,3.435132,0.406905,03:23
1,3.003881,3.402248,0.425401,03:23


In [67]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.955277,3.384412,0.421702,03:11
1,2.920078,3.420116,0.432799,03:25


In [68]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.773730,3.366783,0.422935,03:31
1,2.716070,3.272135,0.430333,03:17


In [69]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.709145,3.263555,0.425401,03:11
1,2.623370,3.233936,0.442663,03:25


In [70]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.640454,3.304458,0.434032,03:32
1,2.545503,3.220748,0.445129,03:33
2,2.553719,3.217932,0.441430,03:15
3,2.503686,3.257223,0.440197,03:31


In [71]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.476148,3.269062,0.441430,03:22
1,2.535405,3.315724,0.445129,03:26
2,2.469517,3.283018,0.447596,03:25
3,2.346374,3.190722,0.450062,03:28


In [72]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.435095,3.305438,0.442663,03:21
1,2.444259,3.257679,0.434032,03:15
2,2.380251,3.281825,0.451295,03:31
3,2.293991,3.293560,0.445129,03:20


In [73]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.262339,3.213201,0.452528,03:33
1,2.400016,3.357565,0.437731,03:32
2,2.319454,3.314281,0.446363,03:15
3,2.324536,3.284770,0.447596,03:18


In [74]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.327006,3.348517,0.451295,03:10
1,2.319893,3.394671,0.457460,03:26
2,2.296823,3.359394,0.443896,03:24
3,2.257173,3.393899,0.451295,03:11


In [75]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.269611,3.369956,0.452528,03:23
1,2.334308,3.385359,0.447596,03:19
2,2.251771,3.336615,0.459926,03:13
3,2.200171,3.385343,0.458693,03:22


In [76]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.202415,3.396734,0.450062,03:27
1,2.287647,3.491828,0.440197,03:26
2,2.256081,3.418550,0.457460,03:09
3,2.200475,3.388902,0.454994,03:11


In [0]:
# save the best model

learn.save_encoder('ask-8-tuning-bigtweet')

# Part three: Predict on the test dataset

In [0]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [79]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

866
866
[76, 18, 1026, 49, 49, 49, 49, 20, 403, 517, 491, 491, 584, 31, 197, 736, 284, 487, 741, 864, 1035, 1035, 1035, 427, 325, 517, 49, 517, 805, 427, 427, 53, 53, 53, 53, 53, 228, 896, 896, 726, 896, 76, 76, 111, 111, 896, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 686, 896, 896, 80, 80, 80, 80, 344, 896, 228, 896, 896, 228, 94, 94, 94, 896, 896, 896, 896, 896, 94, 94, 94, 94, 146, 1035, 211, 471, 94, 492, 896, 337, 517, 265, 265, 265, 276, 517, 896, 1035, 864, 864, 289, 1035, 172, 289, 864, 864, 864, 864, 864, 864, 289, 289, 896, 278, 810, 777, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 289, 127, 343, 355, 94, 98, 745, 284, 89, 896, 473, 447, 210, 896, 896, 896, 1035, 1035, 1002, 165, 896, 896, 210, 182, 30, 289, 864, 896, 76, 896, 874, 864, 864, 864, 864, 704, 289, 513, 183, 183, 183, 183, 183, 183, 183, 896, 896, 289, 1035

In [80]:
correct_num = 0

for i in range(0, 866):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

195
0.2251732101616628
